In [16]:
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd

config database

In [17]:
username = 'postgres'  
password = '2785'
host = 'localhost'
port = '5432'             
database = 'university_database'

Kết nối đến db

In [18]:
conn = psycopg2.connect(user = username, password = password, host = host, port = port, database = database)
cur = conn.cursor()

Đọc dữ liệu

In [19]:
universities = pd.read_csv('../data/processed/university_hcm.csv')
benchmark_scores = pd.read_csv('../data/processed/benchmark_scores_2021_2023_cleaned.csv')
subject_groups = pd.read_csv('../data/processed/subject_groups.csv')
majors = pd.read_csv('../data/processed/majors.csv')

In [20]:
# Danh sách các trường đại học
data_universities = universities.values.tolist()
query_universities = """
    INSERT INTO universities (university_code, university_name, university_url)
    VALUES %s
"""

# Điểm chuẩn
data_benchmark_scores = benchmark_scores.values.tolist()
query_benchmark_scores = """
    INSERT INTO benchmark_scores (year, university_code,program_type,uni_major_code,major_code,major_name,subject_group_code,admission_method,score_scale,benchmark_score)
    VALUES %s
"""

# Các môn tổ hợp đầy đủ
data_subject_groups = subject_groups.values.tolist()
query_subject_groups = """
    INSERT INTO subject_groups (subject_group_code,subject_1,subject_2,subject_3)
    VALUES %s
"""

# Mã ngành
data_majors = majors.values.tolist()
query_majors = """
    INSERT INTO majors (major_code,major_name)
    VALUES %s
"""

Thực hiên INSERT

In [21]:
# trường đại học
execute_values(cur, query_universities,data_universities )
conn.commit()

# tổ hợp môn
execute_values(cur, query_subject_groups,data_subject_groups)
conn.commit()

# mã ngành
execute_values(cur, query_majors,data_majors)
conn.commit()


valid_subject_groups = set(subject_groups['subject_group_code'].unique())
valid_majors = set(majors['major_code'].unique())

# Giữ lại những dòng có subject_group_code và mã ngành hợp lệ
df_benchmark_scores = benchmark_scores[
    (benchmark_scores['subject_group_code'].isin(valid_subject_groups)) &( benchmark_scores['major_code'].isin(valid_majors))
]

data_benchmark_scores = df_benchmark_scores.values.tolist()

# điểm chuẩn
execute_values(cur, query_benchmark_scores, data_benchmark_scores)
conn.commit()

In [22]:
cur.close()
conn.close()